In [1]:
from graphql import graphql_sync
from ariadne import ObjectType, QueryType, make_executable_schema

type_defs = """
    type Query {
        test: Custom
    }

    type Custom {
        node: String
        default: String
    }
"""

query = QueryType()
query.set_field("test", lambda *_: {"node": "custom", "default": "ok"})

custom = ObjectType("Custom")
custom.set_field("node", lambda *_: "deep")

schema = make_executable_schema(type_defs, [query, custom])

result = graphql_sync(schema, "{ test { node default } }")
# assert result.errors is None
# assert result.data == {"test": {"node": "deep", "default": "ok"}}
result.data

{'test': {'node': 'deep', 'default': 'ok'}}

In [2]:
import json

def building_with_id(_, info, _id):
    with open('./buildings.json') as file:
        data = json.load(file)
        for building in data['buildings']:
            if building['id'] == _id:
                return building

def resolve_residents_in_building(building, info):
    print(building)
    with open('./residents.json') as file:
        data = json.load(file)
        residents = [
            resident 
            for resident 
            in data['residents'] 
            if resident['building'] 
            == building['id']]
        return residents

In [3]:
rec=building_with_id(None,None,"1")
# print(rec)
resolve_residents_in_building(rec, None)

{'id': '1', 'buildYear': 2009}


[{'id': '1', 'name': 'Dino', 'age': 56, 'family': ['2', '3'], 'building': '1'},
 {'id': '2', 'name': 'Dina', 'age': 52, 'family': ['1', '3'], 'building': '1'},
 {'id': '3', 'name': 'Leo', 'age': 10, 'family': ['1', '2'], 'building': '1'},
 {'id': '4', 'name': 'Sarah', 'age': 24, 'family': ['5'], 'building': '1'},
 {'id': '5', 'name': 'Harold', 'age': 22, 'family': ['4'], 'building': '1'}]

In [4]:
type_defs='''
type Query {
    building_with_id(_id: ID!): Building
    resident_with_id(_id: ID!): Resident
}

type Building {
    id: ID!
    buildYear: String!
    residents: [Resident]
}

type Resident {
    id: ID!
    name: String!
    age: Int!
    family: [Resident]
    building: Building
}
'''

In [5]:
from graphql import graphql_sync
from ariadne import make_executable_schema, load_schema_from_path, ObjectType, QueryType

# type_defs = load_schema_from_path('schema.graphql')

query = QueryType()
building = ObjectType('Building')
resident = ObjectType('Resident')

query.set_field('building_with_id', building_with_id)
building.set_field('residents', resolve_residents_in_building)

schema = make_executable_schema(type_defs, [building, resident, query])

q="""{
    building_with_id(_id:"1"){
        id
        residents {
            id
            name
        }
    }
}
"""
result = graphql_sync(schema, q)
result.data

{'id': '1', 'buildYear': 2009}


{'building_with_id': {'id': '1',
  'residents': [{'id': '1', 'name': 'Dino'},
   {'id': '2', 'name': 'Dina'},
   {'id': '3', 'name': 'Leo'},
   {'id': '4', 'name': 'Sarah'},
   {'id': '5', 'name': 'Harold'}]}}

In [6]:
result.data['building_with_id']['id']

'1'